In [1]:
import arcpy
from pathlib import Path
import sys

from arcgis.features import GeoAccessor
from ba_data_paths import ba_data

sys.path.append('../../src')
from geoai_retail import utils
from geoai_retail import proximity_local as proximity

sys.path.append('../../scripts')
import data_sources as data

%load_ext autoreload
%autoreload 2

In [2]:
study_bg_id = 530330067003

In [20]:
lyr_bg = ba_data.layer_block_group
print(f'features in layer {int(arcpy.management.GetCount(lyr_bg)[0]):,}')
arcpy.management.SelectLayerByAttribute(lyr_bg, where_clause=f"ID = '{study_bg_id}'")
print(f'features selected in layer {arcpy.management.GetCount(lyr_bg)[0]}')

features in layer 217,195
features selected in layer 1


In [21]:
study_bg_df = GeoAccessor.from_featureclass(lyr_bg)
study_bg_df.spatial.project(4326)
study_bg_df

,ObjectID,ID,NAME,SHAPE
0,207127,530330067003,530330067.003,"{""rings"": [[[-122.34347000016042, 47.630590000..."


In [22]:
webmap = study_bg_df.spatial.plot()
webmap.basemap = 'dark-gray-vector'
webmap

MapView(layout=Layout(height='400px', width='100%'))

In [23]:
store_raw_df = GeoAccessor.from_featureclass(data.destination_store_locations)
store_raw_df.spatial.plot(webmap)
webmap.extent = store_raw_df.spatial.full_extent
store_raw_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,NEAR_FID,NEAR_DIST,SHAPE
0,1,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,...,,6,PointAddress,M,100.0000,INFOGROUP,0,3,0.000021,"{""x"": -122.29909999999995, ""y"": 47.05410000000..."
1,2,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,TACOMA,WA,Washington,98498,6154,44413005,...,,3,StreetAddress,M,96.3594,INFOGROUP,0,35,0.000010,"{""x"": -122.54489999999998, ""y"": 47.18000000000..."
2,3,405129289,GIG HARBOR ACE HARDWARE,POINT FOSDICK DR NW,GIG HARBOR,WA,Washington,98335,1711,44413005,...,,7,StreetAddress,M,100.0000,INFOGROUP,0,41,0.000019,"{""x"": -122.57929999999999, ""y"": 47.30130000000..."
3,4,216082099,SOUTH END ACE HARDWARE,PACIFIC AVE S,SPANAWAY,WA,Washington,98387,8395,44413005,...,,5,PointAddress,M,100.0000,INFOGROUP,0,96,55.586255,"{""x"": -122.43459999999999, ""y"": 47.08740000000..."
4,5,721714069,ACE HARDWARE,112TH ST S,PARKLAND,WA,Washington,98444,5711,44413005,...,,3,PointAddress,M,95.4063,INFOGROUP,0,98,0.000010,"{""x"": -122.43199999999996, ""y"": 47.15610000000..."


In [24]:
bg_df = proximity.prep_sdf_for_nearest(study_bg_df, data.origin_customer_area_id_field)
bg_df

,ID,Name,SHAPE
0,530330067003,530330067003,"{""x"": -122.34790067967894, ""y"": 47.63146443993..."


In [25]:
store_df = proximity.prep_sdf_for_nearest(store_raw_df, data.destination_store_id_field)
store_df.head()

,ID,Name,SHAPE
0,371889957,371889957,"{""x"": -122.29909999999995, ""y"": 47.05410000000..."
1,460556608,460556608,"{""x"": -122.54489999999998, ""y"": 47.18000000000..."
2,405129289,405129289,"{""x"": -122.57929999999999, ""y"": 47.30130000000..."
3,216082099,216082099,"{""x"": -122.43459999999999, ""y"": 47.08740000000..."
4,721714069,721714069,"{""x"": -122.43199999999996, ""y"": 47.15610000000..."


In [26]:
closest_df_raw = proximity._get_closest_df_arcpy(
    origin_df=bg_df,
    dest_df=store_df,
    dest_count=6,
    network_dataset=ba_data.usa_network_dataset,
)
closest_df_raw

,ObjectID,FacilityID,FacilityRank,Name,IncidentCurbApproach,FacilityCurbApproach,IncidentID,Total_TravelTime,Total_Miles,Total_Kilometers,SHAPE
0,1,718542034,1,530330067003 - 718542034,1,2,530330067003,7.751297,1.187608,1.911270,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
1,2,425477834,2,530330067003 - 425477834,1,2,530330067003,9.923221,1.582194,2.546295,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
2,3,426576447,3,530330067003 - 426576447,1,2,530330067003,9.923221,1.582194,2.546295,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
3,4,174746248,4,530330067003 - 174746248,1,2,530330067003,14.874798,3.612485,5.813731,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
4,5,714937939,5,530330067003 - 714937939,1,1,530330067003,21.076403,5.995223,9.648376,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
5,6,402895226,6,530330067003 - 402895226,1,1,530330067003,31.451995,12.124657,19.512743,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."


In [27]:
closest_df = proximity.reformat_closest_result_dataframe(closest_df_raw)
closest_df.spatial.plot(map_widget=webmap, renderer_type='u', col='destination_rank')
webmap.extent = closest_df.spatial.full_extent
closest_df

,origin_id,destination_rank,destination_id,proximity_traveltime,proximity_kilometers,SHAPE
0,530330067003,1,718542034,7.751297,1.911270,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
1,530330067003,2,425477834,9.923221,2.546295,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
2,530330067003,3,426576447,9.923221,2.546295,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
3,530330067003,4,174746248,14.874798,5.813731,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
4,530330067003,5,714937939,21.076403,9.648376,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."
5,530330067003,6,402895226,31.451995,19.512743,"{""hasM"": true, ""paths"": [[[-122.34762999999998..."


In [28]:
closest_df = proximity.explode_closest_rank_dataframe(closest_df)
closest_df

,origin_id,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,destination_id_02,proximity_traveltime_02,proximity_kilometers_02,destination_id_03,proximity_traveltime_03,proximity_kilometers_03,destination_id_04,proximity_traveltime_04,proximity_kilometers_04,destination_id_05,proximity_traveltime_05,proximity_kilometers_05,destination_id_06,proximity_traveltime_06,proximity_kilometers_06
0,530330067003,718542034,7.751297,1.91127,425477834,9.923221,2.546295,426576447,9.923221,2.546295,174746248,14.874798,5.813731,714937939,21.076403,9.648376,402895226,31.451995,19.512743


In [14]:
%%time
closest_df = proximity.closest_dataframe_from_origins_destinations(
    origins=study_bg_df,
    origin_id_fld=data.origin_customer_area_id_field,
    destinations=data.destination_store_locations,
    dest_id_fld=data.destination_store_id_field,
    network_dataset=ba_data.usa_network_dataset,
    destination_count=6,
)

Wall time: 14.9 s


In [15]:
closest_df

,origin_id,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,destination_id_02,proximity_traveltime_02,proximity_kilometers_02,destination_id_03,proximity_traveltime_03,proximity_kilometers_03,destination_id_04,proximity_traveltime_04,proximity_kilometers_04,destination_id_05,proximity_traveltime_05,proximity_kilometers_05,destination_id_06,proximity_traveltime_06,proximity_kilometers_06
0,530330067003,718542034,7.751297,1.91127,425477834,9.923221,2.546295,426576447,9.923221,2.546295,174746248,14.874798,5.813731,714937939,21.076403,9.648376,402895226,31.451995,19.512743


In [16]:
print(f'block group count - {int(arcpy.management.GetCount(data.origin_customer_areas)[0]):,}')
print(f'ace hardware count - {int(arcpy.management.GetCount(data.destination_store_locations)[0]):,}')
print(f'competition count - {int(arcpy.management.GetCount(data.destination_competition_locations)[0]):,}')

block group count - 2,476
ace hardware count - 36
competition count - 308


In [18]:
import pandas as pd

closest_store_df = pd.read_csv('../../data/interim/closest_store.csv', index_col=0)
closest_store_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2473 entries, 0 to 2472
Data columns (total 19 columns):
origin_id                  2473 non-null int64
destination_id_01          2473 non-null int64
proximity_traveltime_01    2473 non-null float64
proximity_kilometers_01    2473 non-null float64
destination_id_02          2473 non-null int64
proximity_traveltime_02    2473 non-null float64
proximity_kilometers_02    2473 non-null float64
destination_id_03          2473 non-null int64
proximity_traveltime_03    2473 non-null float64
proximity_kilometers_03    2473 non-null float64
destination_id_04          2473 non-null int64
proximity_traveltime_04    2473 non-null float64
proximity_kilometers_04    2473 non-null float64
destination_id_05          2473 non-null int64
proximity_traveltime_05    2473 non-null float64
proximity_kilometers_05    2473 non-null float64
destination_id_06          2473 non-null int64
proximity_traveltime_06    2473 non-null float64
proximity_kilometers_06  

In [19]:
closest_store_df.head()

,origin_id,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,destination_id_02,proximity_traveltime_02,proximity_kilometers_02,destination_id_03,proximity_traveltime_03,proximity_kilometers_03,destination_id_04,proximity_traveltime_04,proximity_kilometers_04,destination_id_05,proximity_traveltime_05,proximity_kilometers_05,destination_id_06,proximity_traveltime_06,proximity_kilometers_06
0,530530701003,677129595,43.576384,38.513638,371889957,46.482533,42.526899,427271369,52.472263,44.462895,421027779,55.156955,44.748810,721714069,61.428125,50.456415,216082099,58.905164,54.642485
1,530530714071,371889957,9.405840,6.295963,216082099,13.296163,10.944308,421027779,16.006815,12.366455,721714069,20.814301,15.290477,677129595,29.198017,18.116390,460556608,35.087418,25.237232
2,530530714072,371889957,10.003444,6.125873,216082099,11.536480,8.195827,421027779,18.268181,13.177555,721714069,22.034957,15.623036,677129595,29.203660,20.609116,460556608,34.472733,25.286682
3,530530714073,216082099,12.308627,7.540147,371889957,13.269904,8.468519,421027779,20.223521,13.544703,721714069,23.035677,15.164948,677129595,31.159000,20.976263,460556608,35.244881,24.631002
4,530530714112,216082099,5.292562,3.556479,371889957,12.285190,9.615196,721714069,16.478653,11.395147,421027779,19.885017,15.617594,460556608,28.228815,20.647334,677129595,30.820496,23.049155
